In [1]:
%load_ext autoreload
%autoreload 2
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "1"
import torch
from torch import nn
import numpy as np
from tqdm.auto import tqdm
from matplotlib import pyplot as plt
from sparse_causal_model_learner_rl.learners import rl_learner
from sparse_causal_model_learner_rl.config import Config
from sparse_causal_model_learner_rl.sacred_gin_tune.sacred_wrapper import load_config_files
import gin
import ray
import pickle
from causal_util.helpers import CPU_Unpickler
gin.enter_interactive_mode()

In [2]:
load_config_files(['../vectorincrement/config/ve5_toy_digits.gin',
                   '../sparse_causal_model_learner_rl/configs/ve5_digits_rec.gin',
                   '../sparse_causal_model_learner_rl/configs/server_collect_big.gin'])

['ve5_toy_digits', 've5_digits_rec', 'server_collect_big']

In [3]:
gin.bind_parameter('Config.load_new_config', True)
gin.bind_parameter('Config.load_checkpoint', '/home/sergei/ray_results/ve5_toy_digits_ve5_digits_rec_server_collect_big/main_fcn_98f13_00000_0_2021-02-20_08-59-46/checkpoint_5000/')

In [4]:
ray.init('10.90.40.13:6379')

2021-02-20 10:10:48,613	INFO worker.py:651 -- Connecting to existing Ray cluster at address: 10.90.40.13:6379


{'node_ip_address': '10.90.40.13',
 'raylet_ip_address': '10.90.40.13',
 'redis_address': '10.90.40.13:6379',
 'object_store_address': '/tmp/ray/session_2021-02-20_08-11-18_390441_20021/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-02-20_08-11-18_390441_20021/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-02-20_08-11-18_390441_20021',
 'metrics_export_port': 60463,
 'node_id': '54ef4680490a04e32abf625d1245b34633c06d30'}

In [5]:
f = open(gin.query_parameter('Config.load_checkpoint') + '/checkpoint', 'rb')
#l = CPU_Unpickler(f).load()
l = pickle.load(f)

2021-02-20 10:10:51,763	WARNING session.py:20 -- Session not detected. You should not be calling this function outside `tune.run` or while using the class API. 


Selecting name 878f63f6-735b-11eb-927a-90e2baa15230
Make environment VectorIncrement-v0 None {}
(pid=12371) Make environment VectorIncrement-v0 None {}
(pid=12377) Make environment VectorIncrement-v0 None {}
(pid=12407) Make environment VectorIncrement-v0 None {}
(pid=12404) Make environment VectorIncrement-v0 None {}
(pid=12380) Make environment VectorIncrement-v0 None {}
(pid=12392) Make environment VectorIncrement-v0 None {}
(pid=12388) Make environment VectorIncrement-v0 None {}
(pid=12397) Make environment VectorIncrement-v0 None {}
(pid=12372) Make environment VectorIncrement-v0 None {}
(pid=12391) Make environment VectorIncrement-v0 None {}
(pid=12374) Make environment VectorIncrement-v0 None {}
(pid=12384) Make environment VectorIncrement-v0 None {}
(pid=12440) Make environment VectorIncrement-v0 None {}
(pid=12401) Make environment VectorIncrement-v0 None {}


(pid=12410) Make environment VectorIncrement-v0 None {}


  0.2403024  0.         0.2393224  0.67152673 0.2393224  0.
  0.24321243 0.6725167  0.24321243 0.         0.24172242 0.67016673
  0.24172242]
 [0.4615746  0.31780317 0.6784168  0.         0.46928468 0.3105731
  0.68570685 0.         0.4702447  0.3111931  0.6853269  0.
  0.46620467 0.3107331  0.68587685 0.         0.46331462 0.31412315
  0.6824368 ]
 [0.6821668  0.55856556 0.6821968  0.         0.6894169  0.55086553
  0.6894269  0.         0.68874687 0.5504555  0.6888069  0.
  0.6891769  0.55385554 0.6892669  0.         0.68583685 0.55580556
  0.68587685]
 [0.60197604 0.31780317 0.5221552  0.         0.6104661  0.3105731
  0.5284353  0.         0.6088261  0.3111931  0.5298753  0.
  0.6068861  0.3107331  0.5288653  0.         0.6061361  0.31412315
  0.52433527]
 [0.22438224 0.6657867  0.24079241 0.         0.22385223 0.67297673
  0.2403024  0.         0.22198223 0.6714667  0.2393224  0.
  0.22637226 0.6724267  0.24321243 0.         0.22597227 0.6701267
  0.24172242]] [[0.42744085 0.47157

ERROR:root:Cannot collect data Use collect_and_get_context



[False, False, False, False]
[26, 100, 50, 10, 1] 10
[False, False, False, False, False]
[95, 128, 64, 64, 64, 1] 8
[False, False, False, False]
[8, 128, 64, 32, 1] 95


In [6]:
losses = l.config.get('losses')
execution = {key: (losses[key]['fcn'], losses[key]['coeff']) for key in l.config.get('execution')['opt1']}

In [7]:
np.sum([np.prod(x.shape) for x in l.params_for_optimizers['opt1']])

1408999

In [26]:
losses_vals = {}
grads = {}

def get_grads():
    grads_now = []
    for p in l.params_for_optimizers['opt1']:
        grads_now.append(p.grad)
    return [x.clone() for x in grads_now]

In [27]:
ctx = l.collect_and_get_context()
for key, (loss, coeff) in execution.items():
    loss_local_cache = {}

    l.optimizer_objects['opt1'].zero_grad()
    
    loss_value = loss(**ctx, loss_local_cache=loss_local_cache)
    print(key, loss_value['loss'].item())
    loss_value['loss'].backward()
    losses_vals[key] = loss_value['loss'].item()
    grads[key] = get_grads()
#     l.optimizer_objects['opt1'].zero_grad()
#     l.optimizer_objects['opt1'].step()

fit 82.27526092529297
sparsity 65.08255004882812
fit_all_on 47.05629348754883
reconstruction 12.040839195251465
fit_all_half 82.28622436523438


In [41]:
def tensor_list_shape(lst):
    return [x.shape for x in lst]

def tensor_list_flatten(lst):
    return torch.cat([x.flatten() for x in lst])

def tensor_list_unflatten(t, shape):
    tensors = []
    used_n = 0
    
    for s in shape:
        curr_n = np.prod([int(t) for t in s])
        
        tensors.append(t[used_n:used_n+curr_n].reshape(s))
        
        used_n += curr_n
    
    return tensors

In [46]:
grad_shape = tensor_list_shape(list(grads.values())[0])

In [50]:
grads_flat = {key: tensor_list_flatten(val) for key, val in grads.items()}

In [44]:
# tensor_list_unflatten(tensor_list_flatten(grads['fit']), tensor_list_shape(grads['fit']))

In [28]:
for key, g in grads.items():
    print(key, [torch.norm(z).item() for z in g])

fit [3.1575803756713867, 1.3072092533111572, 16.386396408081055, 12.420012474060059, 2.912383556365967, 19.64764404296875, 5.374879837036133, 13.120229721069336, 9.802406311035156, 5.946949005126953, 2.13551664352417, 3.715576171875, 0.3917442858219147, 2.9974591732025146, 0.4960017204284668, 2.398162603378296, 0.6948057413101196, 2.164566993713379, 0.9753317832946777, 3.203207015991211, 2.051476240158081, 0.6486425399780273, 0.46315035223960876, 0.9855722188949585, 0.4878474473953247, 1.285287857055664, 0.28895869851112366, 2.6879374980926514, 0.36569875478744507, 11.045938491821289, 0.801618218421936, 1.7746752500534058, 1.8787730932235718]
sparsity [0.0, 0.0, 11.401754379272461, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
fit_all_on [2.6835525035858154, 1.5515586137771606, 0.0, 21.306848526000977, 3.59352707862854, 35.542903900146484, 7.13682746887207, 25.3300724029541, 15.43465